In [206]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 
import math

In [207]:
#  Import and read the charity_data.csv.
df = pd.read_csv("Resources/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [208]:
#drop nonmeaningful variables
df = df.drop("EIN",axis=1).drop("NAME",axis=1)
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [209]:
#number of unique values for each variable
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [210]:
#number of instances of each value for variables with at least 10 distinct values
print(df["APPLICATION_TYPE"].value_counts())
print(df["CLASSIFICATION"].value_counts())
print(df["ASK_AMT"].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [211]:
# APPLICATION_TYPEs with fewer than 15 instances are lumped into other
binneddf = df
counts = df["APPLICATION_TYPE"].value_counts()
for APPLICATION_TYPE in df["APPLICATION_TYPE"].unique():
    if counts[APPLICATION_TYPE] < 15:
        binneddf["APPLICATION_TYPE"] = binneddf["APPLICATION_TYPE"].replace(APPLICATION_TYPE, "other")

binneddf["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
other       11
Name: APPLICATION_TYPE, dtype: int64

In [212]:
# CLASSIFICATIONs with fewer than 100 instances are lumped into other
counts = df["CLASSIFICATION"].value_counts()
for CLASSIFICATION in df["CLASSIFICATION"].unique():
    if counts[CLASSIFICATION] < 100:
        binneddf["CLASSIFICATION"] = binneddf["CLASSIFICATION"].replace(CLASSIFICATION, "other")

binneddf["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [213]:
#look at the dataframe so far
binneddf

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [214]:
# turn categorical data into numerical data
dummied = pd.get_dummies(binneddf)
dummied

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [215]:
# split and scale the data
X = dummied.drop("IS_SUCCESSFUL",axis=1)
y = dummied["IS_SUCCESSFUL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [216]:
#check the number of X columns
X_train_scaled.shape

(25724, 53)

In [217]:
# define the topology of the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=53, activation="relu", input_dim=53))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_88 (Dense)            (None, 53)                2862      
                                                                 
 dense_89 (Dense)            (None, 20)                1080      
                                                                 
 dense_90 (Dense)            (None, 12)                252       
                                                                 
 dense_91 (Dense)            (None, 1)                 13        
                                                                 
Total params: 4,207
Trainable params: 4,207
Non-trainable params: 0
_________________________________________________________________


In [218]:
# make a function to save checkpoints
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints", save_weights_only=True,verbose=1)


In [219]:
#train the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
802/804 [============================>.] - ETA: 0s - loss: 0.5733 - accuracy: 0.7202
Epoch 1: saving model to checkpoints
804/804 [==============================] - 1s 975us/step - loss: 0.5732 - accuracy: 0.7203
Epoch 2/100
802/804 [============================>.] - ETA: 0s - loss: 0.5534 - accuracy: 0.7300
Epoch 2: saving model to checkpoints
804/804 [==============================] - 1s 977us/step - loss: 0.5533 - accuracy: 0.7299
Epoch 3/100
774/804 [===========================>..] - ETA: 0s - loss: 0.5502 - accuracy: 0.7324
Epoch 3: saving model to checkpoints
804/804 [==============================] - 1s 950us/step - loss: 0.5509 - accuracy: 0.7315
Epoch 4/100
789/804 [============================>.] - ETA: 0s - loss: 0.5490 - accuracy: 0.7327
Epoch 4: saving model to checkpoints
804/804 [==============================] - 1s 989us/step - loss: 0.5484 - accuracy: 0.7334
Epoch 5/100
754/804 [===========================>..] - ETA: 0s - loss: 0.5484 - accuracy: 0.7330
Epo

797/804 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7393
Epoch 37: saving model to checkpoints
804/804 [==============================] - 1s 992us/step - loss: 0.5371 - accuracy: 0.7388
Epoch 38/100
792/804 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7397
Epoch 38: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7390
Epoch 39/100
757/804 [===========================>..] - ETA: 0s - loss: 0.5371 - accuracy: 0.7398
Epoch 39: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7401
Epoch 40/100
799/804 [============================>.] - ETA: 0s - loss: 0.5365 - accuracy: 0.7399
Epoch 40: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7395
Epoch 41/100
779/804 [============================>.] - ETA: 0s - loss: 0.5365 - accuracy: 0.7392
Epoch 41: sav

749/804 [==========================>...] - ETA: 0s - loss: 0.5332 - accuracy: 0.7434
Epoch 73: saving model to checkpoints
804/804 [==============================] - 1s 965us/step - loss: 0.5339 - accuracy: 0.7416
Epoch 74/100
753/804 [===========================>..] - ETA: 0s - loss: 0.5331 - accuracy: 0.7417
Epoch 74: saving model to checkpoints
804/804 [==============================] - 1s 966us/step - loss: 0.5338 - accuracy: 0.7406
Epoch 75/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5347 - accuracy: 0.7410
Epoch 75: saving model to checkpoints
804/804 [==============================] - 1s 960us/step - loss: 0.5339 - accuracy: 0.7413
Epoch 76/100
785/804 [============================>.] - ETA: 0s - loss: 0.5339 - accuracy: 0.7415
Epoch 76: saving model to checkpoints
804/804 [==============================] - 1s 991us/step - loss: 0.5336 - accuracy: 0.7418
Epoch 77/100
791/804 [============================>.] - ETA: 0s - loss: 0.5343 - accuracy: 0.7413
Epoch 7

In [220]:
#assess the model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
nn_model.save("AlphabetSoupCharity.h5")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7265 - 241ms/epoch - 900us/step
Loss: 0.5563595294952393, Accuracy: 0.7265306115150452
